In [1]:
from tracking import Tracks
import cv2
from utils.metrics import bbox_iou
from utils.plots import draw_boxes
import numpy as np
import torch

/Users/maochenhui/miniforge3/envs/DL/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
with open("./runs/detect/exp/labels/labels.txt", "r") as fo:
    contents = fo.readlines()
fo.close()
contents = [content.strip().split() for content in contents]
contents

[['9', '0', '0.36', '0', '8', '18', '180'],
 ['58', '0', '0.35', '4', '20', '45', '381'],
 ['59', '0', '0.33', '3', '18', '46', '385'],
 ['60', '0', '0.33', '1', '21', '50', '385'],
 ['61', '0', '0.39', '0', '26', '64', '404'],
 ['62', '0', '0.46', '3', '29', '67', '402'],
 ['63', '0', '0.49', '6', '29', '71', '404'],
 ['64', '0', '0.55', '3', '3', '81', '416'],
 ['65', '0', '0.59', '3', '4', '81', '416'],
 ['66', '0', '0.51', '13', '15', '81', '416'],
 ['67', '0', '0.55', '13', '17', '87', '416'],
 ['68', '0', '0.59', '13', '18', '85', '416'],
 ['69', '0', '0.63', '12', '6', '95', '416'],
 ['70', '0', '0.56', '13', '19', '97', '416'],
 ['71', '0', '0.43', '16', '4', '108', '416'],
 ['72', '0', '0.33', '21', '0', '115', '416'],
 ['73', '0', '0.35', '31', '6', '117', '416'],
 ['74', '0', '0.33', '33', '17', '128', '416'],
 ['75', '0', '0.53', '37', '23', '133', '416'],
 ['76', '0', '0.64', '36', '55', '146', '412'],
 ['77', '0', '0.65', '36', '60', '150', '408'],
 ['78', '0', '0.64', '5

In [3]:
bboxes_dict = {}
conf_dict = {}

for i, content in enumerate(contents):
    boxes = []
    confs = []
    for i in range(len(content)//7):
        tmp = content[i*7:(i+1)*7]
        frame_id = int(tmp[0])
        boxes.append([int(b) for b in tmp[3:]])
        confs.append(float(tmp[2]))
    boxes = torch.tensor(boxes)
    bboxes_dict[frame_id] = boxes
    conf_dict[frame_id] = confs

In [4]:
from IPython.core.display import display
from utils.inference_utils import show_array
import IPython
import time
from models.cnn3d import get_model, resnet10

source = "./runs/detect/exp/FALL_DEMO.mp4"
v_cap = cv2.VideoCapture(source)  # initialize the video capture
frame_count = 0
tracks = Tracks(min_conf=0.5)
imgsz = (416, 416)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
cnn3d = get_model(sample_size=112, sample_duration=10, num_classes=2)
cnn3d.load_state_dict(torch.load("./runs/cnn3d/basemodel/best.pt", map_location=device))

try:
    while True:
        success, frame = v_cap.read()  # read frame from video
        IPython.display.clear_output(wait=True)  # clear the previous frame
        if not success:
            print("1 Detect Finished")
            break
        frame = cv2.resize(frame, imgsz)
        if frame_count in bboxes_dict and frame_count in conf_dict:
            tracks.update(bboxes_dict[frame_count], conf_dict[frame_count], frame)
            for track in tracks.tracks:
                if track.current:
                    track.current = False
                    track.__inference__(cnn3d)
                    track.__draw__(frame)
            
        frame_count+=1
        show_array(frame)
        time.sleep(0.02)
        
except KeyboardInterrupt:
    v_cap.release()
    out.release()
    print("2 Detect Finish")

1 Detect Finished


In [8]:
tracks.tracks[0].fall_history

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
